In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn import ensemble
from sklearn.calibration import CalibratedClassifierCV
from sklearn import linear_model
from sklearn.model_selection import KFold, train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn import svm


import numpy as np

df_all = pd.read_csv('NCAA_Tourney_2002-2016_1.csv')
df = df_all.iloc[:, :-4]
df_out = df_all.iloc[:, -4:]


normolizer = preprocessing.Normalizer().fit(df)
df_norm = normolizer.transform(df)

0.8822844752840939 0.6310218941642358


In [21]:
from sklearn.decomposition import KernelPCA, SparsePCA
pca = PCA()
pca.fit(df_norm)

0.8822844752840939 0.6310218941642358


In [22]:
newData = pd.DataFrame(pca.transform(df_norm))
newData = newData.loc[:, 0:10]

X = newData
y = df_out.loc[:, 'result']
kf = KFold(n_splits=5, shuffle=True)

for train, test in kf.split(X):

    X_train, X_valid, y_train, y_valid = train_test_split(X.loc[train], y.loc[train], test_size=0.2, random_state=42)

    X_test, y_test = X.loc[test], y.loc[test]

    clf = GaussianProcessClassifier(1.0 * RBF(1.0), warm_start=True)
    #clf = linear_model.LogisticRegressionCV(Cs=10, solver='liblinear', max_iter=500, tol=1e-6, scoring='neg_log_loss')

    clf.fit(X_train, y_train)

    sig_clf = CalibratedClassifierCV(clf, method="sigmoid", cv='prefit')
    sig_clf.fit(X_valid, y_valid)

    df_out.loc[test, 'prob1'] = sig_clf.predict_proba(X_test)[:, 0]
    df_out.loc[test, 'prob2'] = sig_clf.predict_proba(X_test)[:, 1]


df_out.loc[:, 'log_loss1'] = df_out.loc[:, 'result'] * np.log(df_out.loc[:, 'prob1']) \
                             + (1 - df_out.loc[:, 'result']) * np.log((1 - df_out.loc[:, 'prob1']))

df_out.loc[:, 'log_loss2'] = df_out.loc[:, 'result'] * np.log(df_out.loc[:, 'prob2']) \
                             + (1 - df_out.loc[:, 'result']) * np.log((1 - df_out.loc[:, 'prob2']))

log_loss_1 = - df_out.loc[:, 'log_loss1'].sum()/ len(df_out)
log_loss_2 = - df_out.loc[:, 'log_loss2'].sum()/ len(df_out)

print(log_loss_1, log_loss_2)

0.8822844752840939 0.6310218941642358


In [8]:
df_out.head()

,result,Season,team1_id,team2_id,prob1,prob2,log_loss1,log_loss2
0,1,2002,1104,1194,1.0,1.0,NaN,NaN
1,1,2002,1112,1364,1.0,1.0,NaN,NaN
2,1,2002,1231,1428,1.0,1.0,NaN,NaN
3,1,2002,1242,1221,1.0,1.0,NaN,NaN
4,1,2002,1245,1329,0.0,0.0,-inf,-inf
